In [1]:
import pandas as pd
import sys
import unicodedata
sys.path.append("..")
import ontology_category as oc
sys.path.pop()
svo = oc.init_svo()
units = pd.read_csv('units.csv')[0].tolist()

In [2]:
indicators = pd.read_csv('wdi_glossary.csv', encoding='ISO-8859-1')

In [3]:
indicators.head()

,Code,Indicator Name,Long definition,Source,Dataset_id,Dimension,Datasource_Name,ddpcube_id,Dimension_Name
0,AG.AGR.TRAC.NO,"Agricultural machinery, tractors",Agricultural machinery refers to the number of...,"Food and Agriculture Organization, electronic ...",60,Series,World Development Indicators,2,Series
1,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...,Fertilizer consumption measures the quantity o...,"Food and Agriculture Organization, electronic ...",60,Series,World Development Indicators,2,Series
2,AG.CON.FERT.ZS,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption measures the quantity o...,"Food and Agriculture Organization, electronic ...",60,Series,World Development Indicators,2,Series
3,AG.LND.AGRI.K2,Agricultural land (sq. km),Agricultural land refers to the share of land ...,"Food and Agriculture Organization, electronic ...",60,Series,World Development Indicators,2,Series
4,AG.LND.AGRI.ZS,Agricultural land (% of land area),Agricultural land refers to the share of land ...,"Food and Agriculture Organization, electronic ...",60,Series,World Development Indicators,2,Series


In [4]:
wdi = pd.DataFrame(indicators[['Indicator Name','Long definition']])

In [5]:
wdi.head()

,Indicator Name,Long definition
0,"Agricultural machinery, tractors",Agricultural machinery refers to the number of...
1,Fertilizer consumption (% of fertilizer produc...,Fertilizer consumption measures the quantity o...
2,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption measures the quantity o...
3,Agricultural land (sq. km),Agricultural land refers to the share of land ...
4,Agricultural land (% of land area),Agricultural land refers to the share of land ...


In [6]:
bulk_words = pd.read_csv('english_insignificant_words.csv', encoding='ISO-8859-1',\
                   header = None).fillna('')

In [7]:
', '.join(bulk_words[0].tolist())

'to, of, in, for, on, with, at, by, from, up, about, into, over, after, would, there, their, the, and, a, that, i, it, not, he, as, you, this, but, his, they, her, she, or, an, will, my, are, is, have, has, had, who, which, where, whom, toward, towards, than, then, since, accordingly, also, besides, comparatively, conversely, equally, further, hence, however, incidentally, instead, meanwhile, namely, next, now, rather, still, then, therefore, undoubtedly, additionally, anyway, certainly, consequently, eslewhere, finally, furthermore, henceforth, in addition, in contrast, indeed, likewise, moreover, nevertheless, nonetheless, otherwise, similarly, subsequently, thereafter, thus, yet'

In [8]:
wdi['indicator_name_parse']=wdi['Indicator Name'].str.replace('\(','')\
                            .str.replace('\)','').str.replace(', ',' ')\
                            .str.replace('; ',' ').str.replace(': ',' ').str.lower()
wdi['indicator_name_parse'].head()

0                      agricultural machinery tractors
1    fertilizer consumption % of fertilizer production
2    fertilizer consumption kilograms per hectare o...
3                             agricultural land sq. km
4                     agricultural land % of land area
Name: indicator_name_parse, dtype: object

In [9]:
for word in bulk_words[0].tolist():
    wdi['indicator_name_parse'] = wdi['indicator_name_parse'].str.replace(' '+word+' ',' ')
    temp = wdi['indicator_name_parse'].str.startswith(word+' ')
    if temp.any():
        wdi.loc[temp,'indicator_name_parse'] = wdi.loc[temp,'indicator_name_parse'].str.split(' ',1)[1]
    temp = wdi['indicator_name_parse'].str.endswith(word+' ')
    if temp.any():
        wdi.loc[temp,'indicator_name_parse'] = wdi.loc[temp,'indicator_name_parse'].str.rsplit(' ',1)[0]
wdi['indicator_name_parse'].head()

0                      agricultural machinery tractors
1       fertilizer consumption % fertilizer production
2    fertilizer consumption kilograms per hectare a...
3                             agricultural land sq. km
4                        agricultural land % land area
Name: indicator_name_parse, dtype: object

In [10]:
def is_number(s):
    try:
        float(s.replace(',',''))
        return True
    except ValueError:
        pass
 
    try:
        unicodedata.numeric(s.replace(',',''))
        return True
    except (TypeError, ValueError):
        pass
 
    return False

In [12]:
categories = ['object','process','property','quantity','state','attribute']
other = ['number','currency','scale','units','range','percent']
for cat in categories+other:
    wdi[cat]=''
for i, row in wdi.iterrows():
    for word in row['indicator_name_parse'].split():
        if is_number(word):
            wdi.loc[i,'number'] = (wdi.loc[i,'number'] + ', '+ word).lstrip(', ')
        elif is_number(word.replace('$','')):
            wdi.loc[i,'currency'] = (wdi.loc[i,'currency'] + ', '+ word).lstrip(', ')
        elif is_number(word.replace('-','')):
            wdi.loc[i,'range'] = (wdi.loc[i,'range'] + ', '+ word).lstrip(', ')
        elif is_number(word.replace('%','')):
            wdi.loc[i,'percent'] = (wdi.loc[i,'percent'] + ', '+ word).lstrip(', ')
        elif '=' in word:
            wdi.loc[i,'scale'] = (wdi.loc[i,'scale'] + ', '+ word).lstrip(', ')
        elif word in units:
            wdi.loc[i,'units'] = (wdi.loc[i,'units'] + ', '+ word).lstrip(', ')
        else:
            for cat in categories:
                if svo.is_cat(word,cat,out='short'):
                    wdi.loc[i,cat] = (wdi.loc[i,cat] + ', '+ word).lstrip(', ')

In [13]:
display(wdi)

,Indicator Name,Long definition,indicator_name_parse,object,process,property,quantity,state,attribute,number,currency,scale
0,"Agricultural machinery, tractors",Agricultural machinery refers to the number of...,agricultural machinery tractors,"machinery, tractors",,,,,agricultural,,,
1,Fertilizer consumption (% of fertilizer produc...,Fertilizer consumption measures the quantity o...,fertilizer consumption % fertilizer production,"fertilizer, fertilizer, production","consumption, production",consumption,production,consumption,,,,
2,Fertilizer consumption (kilograms per hectare ...,Fertilizer consumption measures the quantity o...,fertilizer consumption kilograms per hectare a...,"fertilizer, land","consumption, land","consumption, land","kilograms, hectare","consumption, land",arable,,,
3,Agricultural land (sq. km),Agricultural land refers to the share of land ...,agricultural land sq. km,land,land,land,km,land,agricultural,,,
4,Agricultural land (% of land area),Agricultural land refers to the share of land ...,agricultural land % land area,"land, land, area","land, land","land, land, area",,"land, land, area",agricultural,,,
5,Arable land (hectares),Arable land (in hectares) includes land define...,arable land hectares,land,land,land,hectares,land,arable,,,
6,Arable land (hectares per person),Arable land (hectares per person) includes lan...,arable land hectares per person,"land, person",land,land,hectares,land,arable,,,
7,Arable land (% of land area),Arable land includes land defined by the FAO a...,arable land % land area,"land, land, area","land, land","land, land, area",,"land, land, area",arable,,,
8,Land under cereal production (hectares),Land under cereal production refers to harvest...,land under cereal production hectares,"land, cereal, production","land, production",land,"production, hectares",land,"under, cereal",,,
9,Permanent cropland (% of land area),Permanent cropland is land cultivated with cro...,permanent cropland % land area,"permanent, land, area",land,"land, area",,"land, area",permanent,,,
